In [1]:
from datetime import date
from dateutil.rrule import rrule, MONTHLY
import pandas as pd
from uuid import uuid4
import plotly.express as px

long_df = px.data.medals_long()

In [2]:
def make_timeseries(amount, from_ts, until_ts):
    return {ts: i*amount 
            for i, ts in enumerate(rrule(MONTHLY, dtstart=from_ts, until=until_ts), start=1)}

In [49]:
ts1 = make_timeseries(100, date(2018, 1, 1), date(2019, 1, 1))
ts2 = make_timeseries(200, date(2020, 1, 1), date(2021, 1, 1))

In [50]:
s1 = pd.Series(ts1, name=uuid4())
s2 = pd.Series(ts2, name=uuid4())

In [57]:
pd.DataFrame().join([s1, s2], how="outer").fillna(method="ffill").fillna(0).sum(axis=1)

2018-01-01     100.0
2018-02-01     200.0
2018-03-01     300.0
2018-04-01     400.0
2018-05-01     500.0
2018-06-01     600.0
2018-07-01     700.0
2018-08-01     800.0
2018-09-01     900.0
2018-10-01    1000.0
2018-11-01    1100.0
2018-12-01    1200.0
2019-01-01    1300.0
2020-01-01    1500.0
2020-02-01    1700.0
2020-03-01    1900.0
2020-04-01    2100.0
2020-05-01    2300.0
2020-06-01    2500.0
2020-07-01    2700.0
2020-08-01    2900.0
2020-09-01    3100.0
2020-10-01    3300.0
2020-11-01    3500.0
2020-12-01    3700.0
2021-01-01    3900.0
dtype: float64

In [51]:
df = pd.DataFrame()
df = df.join(-s1, how="outer").asfreq("MS").fillna(method="ffill").fillna(0)
df = df.join(s2, how="outer").asfreq("MS").fillna(method="ffill").fillna(0)

In [37]:
df = df.melt(ignore_index=False)
df.index.name = "Datum"

In [48]:
total = df.sum(axis=1)
total

2018-01-01    -100.0
2018-02-01    -200.0
2018-03-01    -300.0
2018-04-01    -400.0
2018-05-01    -500.0
2018-06-01    -600.0
2018-07-01    -700.0
2018-08-01    -800.0
2018-09-01    -900.0
2018-10-01   -1000.0
2018-11-01   -1100.0
2018-12-01   -1200.0
2019-01-01   -1300.0
2019-02-01   -1300.0
2019-03-01   -1300.0
2019-04-01   -1300.0
2019-05-01   -1300.0
2019-06-01   -1300.0
2019-07-01   -1300.0
2019-08-01   -1300.0
2019-09-01   -1300.0
2019-10-01   -1300.0
2019-11-01   -1300.0
2019-12-01   -1300.0
2020-01-01   -1100.0
2020-02-01    -900.0
2020-03-01    -700.0
2020-04-01    -500.0
2020-05-01    -300.0
2020-06-01    -100.0
2020-07-01     100.0
2020-08-01     300.0
2020-09-01     500.0
2020-10-01     700.0
2020-11-01     900.0
2020-12-01    1100.0
2021-01-01    1300.0
Freq: MS, dtype: float64

In [41]:
int(100 > 0)

1

In [32]:
def calc_interest(series, interest_rate=0.1):
    for i in range(1, len(series)):
        series.iloc[i] = series.iloc[i] + series.iloc[i-1]*interest_rate
    return series

In [35]:
%%timeit
calc_interest(total)

13.8 ms ± 207 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
with_interest = calc_interest(total)
with_interest

,value
Datum,
2018-01-01,100.000000
2018-02-01,220.000000
2018-03-01,343.000000
2018-04-01,466.400000
2018-05-01,589.850000
2018-06-01,713.306000
2018-07-01,836.762700
2018-08-01,960.219480
2018-09-01,1083.676269


In [11]:
fig = px.bar(df, x=df.index, y="value", color="variable")
fig.add_bar(x=total.index, y=total.value, base=False)
fig.show()

ValueError: Value of 'base' is not the name of a column in 'data_frame'. Expected one of ['variable', 'value'] but received: True